In [1]:
import random
import pymysql
from faker import Faker
from datetime import datetime, timedelta
import decimal
fake = Faker('en_US')

In [2]:
# DEPARTMENT
conn = pymysql.connect(
    host='127.0.0.1',
    user='skotam',
    password='skotam',
    database='ABCcompany'
)
cursor = conn.cursor()
cursor.execute("DELETE FROM DEPARTMENT;")
conn.commit()
print("-- INSERT INTO DEPARTMENT Table")
departments = [
    (1, '08:00:00', '17:00:00', 'Marketing'),
    (2, '08:30:00', '17:30:00', 'Engineering'),
    (3, '08:00:00', '16:00:00', 'Human Resources'),
    (4, '09:00:00', '17:00:00', 'Finance')
]
cursor.executemany(
    "INSERT INTO DEPARTMENT (DeptID, StartTime, EndTime, DeptName) VALUES (%s, %s, %s, %s)",
    departments
)
conn.commit()
print(f"Inserted {len(departments)} departments successfully!")
cursor.close()
conn.close()

-- INSERT INTO DEPARTMENT Table
Inserted 4 departments successfully!


In [3]:
# PERSON
conn = pymysql.connect(
    host='127.0.0.1',
    user='skotam',
    password='skotam',
    database='ABCcompany'
)
cursor = conn.cursor()
num_people = 150
cursor.execute("DELETE FROM PERSON;")
conn.commit()
print("-- INSERT INTO PERSON Table")
person_data = []

person_data.append((
    'Hellen', None, 'Cole', random.randint(22, 64), 'Female',
    fake.street_address(), fake.secondary_address(), fake.city(),
    fake.state(), fake.zipcode(), f"{random.randint(100,999)}-{random.randint(100,999)}-{random.randint(1000,9999)}", fake.email()
))

for _ in range(num_people - 1):
    fname = fake.first_name()
    mname = fake.first_name() if random.random() > 0.3 else None
    lname = fake.last_name()
    age = random.randint(18, 64)
    gender = random.choice(['Male', 'Female', 'Other'])
    add1 = fake.street_address()
    add2 = fake.secondary_address() if random.random() > 0.5 else None
    city = fake.city()
    state = fake.state()
    zipcode = fake.zipcode()
    phone = f"{random.randint(100,999)}-{random.randint(100,999)}-{random.randint(1000,9999)}"
    email = fake.email()
    person_data.append((fname, mname, lname, age, gender, add1, add2, city, state, zipcode, phone, email))
cursor.executemany(
    """INSERT INTO PERSON (Fname, Mname, Lname, Age, Gender, AddLine1, AddLine2, 
       City, State, Zipcode, PhoneNum, EmailAdd) 
       VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)""",
    person_data
)
conn.commit()
print(f"Inserted {len(person_data)} people successfully!")
cursor.close()
conn.close()

-- INSERT INTO PERSON Table
Inserted 150 people successfully!


In [4]:
# EMPLOYEE
conn = pymysql.connect(
    host='127.0.0.1',
    user='skotam',
    password='skotam',
    database='ABCcompany'
)
cursor = conn.cursor()
cursor.execute("DELETE FROM EMPLOYEE;")
conn.commit()
print("-- INSERT INTO EMPLOYEE Table")
cursor.execute("SELECT Personal_ID FROM PERSON")
person_ids = [row[0] for row in cursor.fetchall()]
cursor.execute("SELECT DeptID FROM DEPARTMENT")
dept_ids = [row[0] for row in cursor.fetchall()]
employee_ids = random.sample(person_ids, 80)

# define to avoid multiple CEOs
employee_data = []


c_level_roles = [
    ('C-level', 'CEO'),
    ('C-level', 'CFO'),
    ('C-level', 'CTO')
]
for i, (rank, title) in enumerate(c_level_roles):
    emp_id = employee_ids[i]
    salary = round(random.uniform(150000, 200000), 2)
    trans_no = f"TXN{random.randint(10000, 99999)}"
    pay_date = fake.date_between(start_date='-1y', end_date='today')
    dept_id = random.choice(dept_ids)
    employee_data.append((emp_id, rank, title, salary, trans_no, pay_date, dept_id, None))

# limited VPs
for i in range(3, 10):
    emp_id = employee_ids[i]
    rank = 'VP'
    title = random.choice(['Manager', 'Analyst', 'Consultant'])
    salary = round(random.uniform(120000, 150000), 2)
    trans_no = f"TXN{random.randint(10000, 99999)}"
    pay_date = fake.date_between(start_date='-1y', end_date='today')
    dept_id = random.choice(dept_ids)
    employee_data.append((emp_id, rank, title, salary, trans_no, pay_date, dept_id, None))

# remaining employees: Junior, Mid-level, Senior
ranks = ['Junior', 'Mid-level', 'Senior']
titles = ['Engineer', 'Manager', 'Analyst', 'Consultant']
for i in range(10, len(employee_ids)):
    emp_id = employee_ids[i]
    rank = random.choice(ranks)
    title = random.choice(titles)
    

    if rank == 'Junior':
        salary = round(random.uniform(40000, 70000), 2)
    elif rank == 'Mid-level':
        salary = round(random.uniform(70000, 100000), 2)
    else:  # Senior
        salary = round(random.uniform(100000, 130000), 2)
    
    trans_no = f"TXN{random.randint(10000, 99999)}"
    pay_date = fake.date_between(start_date='-1y', end_date='today')
    dept_id = random.choice(dept_ids)
    employee_data.append((emp_id, rank, title, salary, trans_no, pay_date, dept_id, None))

cursor.executemany(
    """INSERT INTO EMPLOYEE (Personal_ID, Ranked, Title, SalaryAmount, 
       SalaryTransactionNo, SalaryPayDate, DeptID, Supervisor_ID) 
       VALUES (%s, %s, %s, %s, %s, %s, %s, %s)""",
    employee_data
)
conn.commit()


employees_with_supervisors = employee_ids[10:]  
for emp_id in employees_with_supervisors:
    supervisor_id = random.choice(employee_ids[:10])  
    cursor.execute(
        "UPDATE EMPLOYEE SET Supervisor_ID = %s WHERE Personal_ID = %s",
        (supervisor_id, emp_id)
    )
conn.commit()
print(f"Inserted {len(employee_data)} employees successfully!")
cursor.close()
conn.close()

-- INSERT INTO EMPLOYEE Table
Inserted 80 employees successfully!


In [5]:
# SALARY
conn = pymysql.connect(
    host='127.0.0.1',
    user='skotam',
    password='skotam',
    database='ABCcompany'
)
cursor = conn.cursor()
cursor.execute("DELETE FROM SALARY;")
conn.commit()
print("-- INSERT INTO SALARY Table")
cursor.execute("SELECT Personal_ID FROM EMPLOYEE")
employee_ids = [row[0] for row in cursor.fetchall()]
salary_data = []
for emp_id in employee_ids:
    num_months = random.randint(3, 12)
    base_salary = round(random.uniform(3000, 15000), 2)
    for i in range(num_months):
        trans_no = f"TXN{emp_id}{i:03d}"
        pay_date = datetime(2010, 1, 1) + timedelta(days=30*i)
        amount = round(base_salary + random.uniform(-500, 500), 2)
        salary_data.append((emp_id, trans_no, pay_date, amount))
cursor.executemany(
    "INSERT INTO SALARY (Personal_ID, TransactionNo, PayDate, Amount) VALUES (%s, %s, %s, %s)",
    salary_data
)
conn.commit()
print(f"Inserted {len(salary_data)} salary records successfully!")
cursor.close()
conn.close()

-- INSERT INTO SALARY Table
Inserted 618 salary records successfully!


In [6]:
# EMP_DEPT_HISTORY
conn = pymysql.connect(
    host='127.0.0.1',
    user='skotam',
    password='skotam',
    database='ABCcompany'
)
cursor = conn.cursor()
cursor.execute("DELETE FROM EMP_DEPT_HISTORY;")
conn.commit()
print("-- INSERT INTO EMP_DEPT_HISTORY Table")
cursor.execute("SELECT Personal_ID FROM EMPLOYEE")
employee_ids = [row[0] for row in cursor.fetchall()]
cursor.execute("SELECT DeptID FROM DEPARTMENT")
dept_ids = [row[0] for row in cursor.fetchall()]
history_data = []
# some employees have worked in multiple departments
for emp_id in random.sample(employee_ids, min(30, len(employee_ids))):
    num_depts = random.randint(2, 4)
    used_depts = random.sample(dept_ids, min(num_depts, len(dept_ids)))
    start_date = datetime(2020, 1, 1)
    for i, dept_id in enumerate(used_depts):
        start_time = start_date + timedelta(days=i*180)
        if i < len(used_depts) - 1:
            end_time = start_date + timedelta(days=(i+1)*180)
        else:
            end_time = None
        history_data.append((emp_id, dept_id, start_time, end_time))
cursor.executemany(
    "INSERT INTO EMP_DEPT_HISTORY (Personal_ID, DeptID, StartTime, EndTime) VALUES (%s, %s, %s, %s)",
    history_data
)
conn.commit()
print(f"Inserted {len(history_data)} department history records successfully!")
cursor.close()
conn.close()


-- INSERT INTO EMP_DEPT_HISTORY Table
Inserted 94 department history records successfully!


In [7]:
# CUSTOMER
conn = pymysql.connect(
    host='127.0.0.1',
    user='skotam',
    password='skotam',
    database='ABCcompany'
)
cursor = conn.cursor()
cursor.execute("DELETE FROM CUSTOMER;")
conn.commit()
print("-- INSERT INTO CUSTOMER Table")
cursor.execute("SELECT Personal_ID FROM PERSON WHERE Personal_ID NOT IN (SELECT Personal_ID FROM EMPLOYEE)")
available_persons = [row[0] for row in cursor.fetchall()]
cursor.execute("SELECT Personal_ID FROM EMPLOYEE")
employee_ids = [row[0] for row in cursor.fetchall()]
customer_ids = random.sample(available_persons, min(40, len(available_persons)))
customer_ids.extend(random.sample(employee_ids, 10))
customer_data = []
for cust_id in customer_ids:
    pref_salesman = random.choice(employee_ids)
    customer_data.append((cust_id, pref_salesman))
cursor.executemany(
    "INSERT INTO CUSTOMER (Personal_ID, PrefSalesmen) VALUES (%s, %s)",
    customer_data
)
conn.commit()
print(f"Inserted {len(customer_data)} customers successfully!")
cursor.close()
conn.close()

-- INSERT INTO CUSTOMER Table
Inserted 50 customers successfully!


In [8]:
# POTENTIAL_EMPLOYEE
conn = pymysql.connect(
    host='127.0.0.1',
    user='skotam',
    password='skotam',
    database='ABCcompany'
)
cursor = conn.cursor()
cursor.execute("DELETE FROM POTENTIAL_EMPLOYEE;")
conn.commit()
print("-- INSERT INTO POTENTIAL_EMPLOYEE Table")
cursor.execute("SELECT Personal_ID FROM PERSON WHERE Personal_ID NOT IN (SELECT Personal_ID FROM EMPLOYEE)")
available_persons = [row[0] for row in cursor.fetchall()]
cursor.execute("SELECT Personal_ID FROM EMPLOYEE LIMIT 10")
some_employees = [row[0] for row in cursor.fetchall()]
potential_emp_ids = random.sample(available_persons, min(30, len(available_persons)))
potential_emp_ids.extend(some_employees)
potential_emp_data = [(pid,) for pid in potential_emp_ids]
cursor.executemany(
    "INSERT INTO POTENTIAL_EMPLOYEE (Personal_ID) VALUES (%s)",
    potential_emp_data
)
conn.commit()
print(f"Inserted {len(potential_emp_data)} potential employees successfully!")
cursor.close()
conn.close()

-- INSERT INTO POTENTIAL_EMPLOYEE Table
Inserted 40 potential employees successfully!


In [9]:
# CANDIDATE
conn = pymysql.connect(
    host='127.0.0.1',
    user='skotam',
    password='skotam',
    database='ABCcompany'
)
cursor = conn.cursor()
cursor.execute("DELETE FROM CANDIDATE;")
conn.commit()
print("-- INSERT INTO CANDIDATE Table")
cursor.execute("SELECT Personal_ID FROM POTENTIAL_EMPLOYEE")
potential_emp_ids = [row[0] for row in cursor.fetchall()]
cursor.execute("SELECT Personal_ID FROM PERSON WHERE Fname = 'Hellen' AND Lname = 'Cole'")
hellen_id = cursor.fetchone()
if hellen_id and hellen_id[0] not in potential_emp_ids:
    cursor.execute("INSERT INTO POTENTIAL_EMPLOYEE (Personal_ID) VALUES (%s)", (hellen_id[0],))
    conn.commit()
    potential_emp_ids.append(hellen_id[0])
candidate_data = [(pid,) for pid in potential_emp_ids]
cursor.executemany(
    "INSERT INTO CANDIDATE (Personal_ID) VALUES (%s)",
    candidate_data
)
conn.commit()
print(f"Inserted {len(candidate_data)} candidates successfully!")
cursor.close()
conn.close()


-- INSERT INTO CANDIDATE Table
Inserted 41 candidates successfully!


In [10]:
# SITE
conn = pymysql.connect(
    host='127.0.0.1',
    user='skotam',
    password='skotam',
    database='ABCcompany'
)
cursor = conn.cursor()
cursor.execute("DELETE FROM SITE;")
conn.commit()
print("-- INSERT INTO SITE Table")
site_data = []
for i in range(1, 16):
    site_name = f"{fake.city()} {random.choice(['Marketing', 'Sales', 'Distribution', 'Retail'])} Center"
    site_location = f"{fake.street_address()}, {fake.city()}, {fake.state()}"
    site_data.append((i, site_name, site_location))
cursor.executemany(
    "INSERT INTO SITE (SiteID, SiteName, SiteLocation) VALUES (%s, %s, %s)",
    site_data
)
conn.commit()
print(f"Inserted {len(site_data)} sites successfully!")
cursor.close()
conn.close()

-- INSERT INTO SITE Table
Inserted 15 sites successfully!


In [11]:
# VENDOR
conn = pymysql.connect(
    host='127.0.0.1',
    user='skotam',
    password='skotam',
    database='ABCcompany'
)
cursor = conn.cursor()
cursor.execute("DELETE FROM VENDOR;")
conn.commit()
print("-- INSERT INTO VENDOR Table")
vendor_data = []
for i in range(1, 21):
    name = fake.company()
    address = f"{fake.street_address()}, {fake.city()}, {fake.state()} {fake.zipcode()}"
    acc_no = f"ACC{random.randint(100000, 999999)}"
    credit_rating = random.choice(['AAA', 'AA', 'A', 'BBB', 'BB', 'B'])
    ws_url = f"https://{fake.domain_name()}/purchasing/api"
    vendor_data.append((i, name, address, acc_no, credit_rating, ws_url))
cursor.executemany(
    "INSERT INTO VENDOR (VendorID, Name, Address, AccNo, CreditRating, PurchasingWSURL) VALUES (%s, %s, %s, %s, %s, %s)",
    vendor_data
)
conn.commit()
print(f"Inserted {len(vendor_data)} vendors successfully!")
cursor.close()
conn.close()

-- INSERT INTO VENDOR Table
Inserted 20 vendors successfully!


In [13]:
# PART 
conn = pymysql.connect(
    host='127.0.0.1',
    user='skotam',
    password='skotam',
    database='ABCcompany'
)
cursor = conn.cursor()
cursor.execute("DELETE FROM PART;")
conn.commit()
print("-- INSERT INTO PART Table")
part_data = []

part_data.append((1, 'Cup', 3.5))

part_names = ['Screw', 'Bolt', 'Handle', 'Panel', 'Screen', 'Button', 'Wire', 'Plate', 
              'Cover', 'Frame', 'Chip', 'Motor', 'Hinge', 'Spring', 'Gear', 'Lens',
              'Switch', 'Socket', 'Bracket', 'Bearing', 'Valve', 'Filter', 'Cable',
              'Sensor', 'Speaker', 'Battery', 'Connector', 'Pad', 'Wheel']
for i, name in enumerate(part_names, start=2):
    weight = round(random.uniform(0.1, 20.0), 2)
    part_data.append((i, name, weight))
cursor.executemany(
    "INSERT INTO PART (PartID, PartName, PartWeight) VALUES (%s, %s, %s)",
    part_data
)
conn.commit()
print(f"Inserted {len(part_data)} parts successfully!")
cursor.close()
conn.close()


-- INSERT INTO PART Table
Inserted 30 parts successfully!


In [14]:
# PRODUCT
conn = pymysql.connect(
    host='127.0.0.1',
    user='skotam',
    password='skotam',
    database='ABCcompany'
)
cursor = conn.cursor()
cursor.execute("DELETE FROM PRODUCT;")
conn.commit()
print("-- INSERT INTO PRODUCT Table")
product_types = ['Electronics', 'Furniture', 'Clothing', 'Sports Equipment', 'Kitchen', 'Office Supplies']
sizes = ['Small', 'Medium', 'Large', 'XL', 'One Size']
styles = ['Modern', 'Classic', 'Vintage', 'Contemporary', 'Traditional']
product_data = []
for i in range(1, 51):
    product_type = random.choice(product_types)
    size = random.choice(sizes)
    list_price = round(random.uniform(50, 500), 2)
    weight = round(random.uniform(0.5, 50), 2)
    style = random.choice(styles)
    product_data.append((i, product_type, size, list_price, weight, style))
cursor.executemany(
    "INSERT INTO PRODUCT (ProductID, ProductType, Size, ListPrice, Weight, Style) VALUES (%s, %s, %s, %s, %s, %s)",
    product_data
)
conn.commit()
print(f"Inserted {len(product_data)} products successfully!")
cursor.close()
conn.close()

-- INSERT INTO PRODUCT Table
Inserted 50 products successfully!


In [15]:
# PRODUCT_PART 
conn = pymysql.connect(
    host='127.0.0.1',
    user='skotam',
    password='skotam',
    database='ABCcompany'
)
cursor = conn.cursor()
cursor.execute("DELETE FROM PRODUCT_PART;")
conn.commit()
print("-- INSERT INTO PRODUCT_PART Table")
cursor.execute("SELECT ProductID FROM PRODUCT")
product_ids = [row[0] for row in cursor.fetchall()]
cursor.execute("SELECT PartID FROM PART")
part_ids = [row[0] for row in cursor.fetchall()]
product_part_data = []
for product_id in product_ids:
    num_parts = random.randint(2, 8)
    parts_used = random.sample(part_ids, min(num_parts, len(part_ids)))
    for part_id in parts_used:
        quantity = random.randint(1, 10)
        product_part_data.append((product_id, part_id, quantity))
cursor.executemany(
    "INSERT INTO PRODUCT_PART (ProductID, PartID, Quantity) VALUES (%s, %s, %s)",
    product_part_data
)
conn.commit()
print(f"Inserted {len(product_part_data)} product-part relationships successfully!")
cursor.close()
conn.close()

-- INSERT INTO PRODUCT_PART Table
Inserted 240 product-part relationships successfully!


In [16]:
# JOB_POSITION
conn = pymysql.connect(
    host='127.0.0.1',
    user='skotam',
    password='skotam',
    database='ABCcompany'
)
cursor = conn.cursor()
cursor.execute("DELETE FROM JOB_POSITION;")
conn.commit()
print("-- INSERT INTO JOB_POSITION Table")
cursor.execute("SELECT DeptID, DeptName FROM DEPARTMENT")
departments = cursor.fetchall()
job_data = []
job_data.append((11111, 'Senior Marketing Analyst', 1, datetime(2011, 1, 15)))
job_data.append((12345, 'Software Engineer', 2, datetime(2010, 12, 1)))
for i in range(3):
    job_data.append((20000 + i, f'Marketing Position {i+1}', 1, datetime(2011, 1, random.randint(1, 31))))
job_id = 1000
for i in range(30):
    dept_id, dept_name = random.choice(departments)
    description = f"{fake.job()} in {dept_name}"
    posted_date = fake.date_between(start_date='-3y', end_date='today')
    job_data.append((job_id, description, dept_id, posted_date))
    job_id += 1
cursor.executemany(
    "INSERT INTO JOB_POSITION (JobID, Description, DeptID, PostedDate) VALUES (%s, %s, %s, %s)",
    job_data
)
conn.commit()
print(f"Inserted {len(job_data)} job positions successfully!")
cursor.close()
conn.close()

-- INSERT INTO JOB_POSITION Table
Inserted 35 job positions successfully!


In [17]:
# JOB_APPLICATION
conn = pymysql.connect(
    host='127.0.0.1',
    user='skotam',
    password='skotam',
    database='ABCcompany'
)
cursor = conn.cursor()
cursor.execute("DELETE FROM JOB_APPLICATION;")
conn.commit()
print("-- INSERT INTO JOB_APPLICATION Table")
cursor.execute("SELECT Personal_ID FROM PERSON")
person_ids = [row[0] for row in cursor.fetchall()]
cursor.execute("SELECT Personal_ID FROM EMPLOYEE")
employee_ids = [row[0] for row in cursor.fetchall()]
cursor.execute("SELECT JobID FROM JOB_POSITION")
job_ids = [row[0] for row in cursor.fetchall()]
application_data = []
applied_combinations = set()
for emp_id in random.sample(employee_ids, min(5, len(employee_ids))):
    apply_date = fake.date_between(start_date='-2y', end_date='today')
    application_data.append((emp_id, 12345, apply_date))
    applied_combinations.add((emp_id, 12345))
for _ in range(150):
    person_id = random.choice(person_ids)
    job_id = random.choice(job_ids)
    if (person_id, job_id) not in applied_combinations:
        apply_date = fake.date_between(start_date='-2y', end_date='today')
        application_data.append((person_id, job_id, apply_date))
        applied_combinations.add((person_id, job_id))
cursor.executemany(
    "INSERT INTO JOB_APPLICATION (Personal_ID, JobID, ApplyDate) VALUES (%s, %s, %s)",
    application_data
)
conn.commit()
print(f"Inserted {len(application_data)} job applications successfully!")
cursor.close()
conn.close()

-- INSERT INTO JOB_APPLICATION Table
Inserted 151 job applications successfully!


In [18]:
# INTERVIEW 
conn = pymysql.connect(
    host='127.0.0.1',
    user='skotam',
    password='skotam',
    database='ABCcompany'
)
cursor = conn.cursor()
cursor.execute("DELETE FROM INTERVIEW;")
conn.commit()
print("-- INSERT INTO INTERVIEW Table")
cursor.execute("SELECT Personal_ID FROM CANDIDATE")
candidate_ids = [row[0] for row in cursor.fetchall()]
cursor.execute("SELECT Personal_ID FROM PERSON WHERE Fname = 'Hellen' AND Lname = 'Cole'")
hellen_result = cursor.fetchone()
hellen_id = hellen_result[0] if hellen_result else None
cursor.execute("SELECT Personal_ID FROM EMPLOYEE")
employee_ids = [row[0] for row in cursor.fetchall()]
cursor.execute("SELECT JobID FROM JOB_POSITION")
job_ids = [row[0] for row in cursor.fetchall()]
interview_data = []

if hellen_id:
    for i in range(6):
        interview_time = datetime(2011, 2, random.randint(1, 28), random.randint(9, 17), 0)
        grade = random.randint(65, 95)
        interviewer_id = random.choice(employee_ids)
        interview_data.append((hellen_id, 11111, interview_time, grade, interviewer_id))

for candidate_id in random.sample(candidate_ids, min(30, len(candidate_ids))):
    job_id = random.choice(job_ids)
    num_interviews = random.randint(3, 8)
    for i in range(num_interviews):
        interview_time = fake.date_time_between(start_date='-1y', end_date='now')
        grade = random.randint(40, 100)
        interviewer_id = random.choice(employee_ids)
        interview_data.append((candidate_id, job_id, interview_time, grade, interviewer_id))
cursor.executemany(
    "INSERT INTO INTERVIEW (Candidate_ID, JobID, InterviewTime, Grade, Interviewer_ID) VALUES (%s, %s, %s, %s, %s)",
    interview_data
)
conn.commit()
print(f"Inserted {len(interview_data)} interviews successfully!")
cursor.close()
conn.close()

-- INSERT INTO INTERVIEW Table
Inserted 175 interviews successfully!


In [19]:
# PURCHASE_FROM
conn = pymysql.connect(
    host='127.0.0.1',
    user='skotam',
    password='skotam',
    database='ABCcompany'
)
cursor = conn.cursor()
cursor.execute("DELETE FROM PURCHASE_FROM;")
conn.commit()
print("-- INSERT INTO PURCHASE_FROM Table")
cursor.execute("SELECT JobID FROM JOB_POSITION")
job_ids = [row[0] for row in cursor.fetchall()]
cursor.execute("SELECT VendorID FROM VENDOR")
vendor_ids = [row[0] for row in cursor.fetchall()]
purchase_data = []
purchased_combinations = set()
for _ in range(50):
    job_id = random.choice(job_ids)
    vendor_id = random.choice(vendor_ids)
    if (job_id, vendor_id) not in purchased_combinations:
        posted_date = fake.date_between(start_date='-2y', end_date='today')
        purchase_data.append((job_id, vendor_id, posted_date))
        purchased_combinations.add((job_id, vendor_id))
cursor.executemany(
    "INSERT INTO PURCHASE_FROM (JobID, VendorID, PostedDate) VALUES (%s, %s, %s)",
    purchase_data
)
conn.commit()
print(f"Inserted {len(purchase_data)} purchase records successfully!")
cursor.close()
conn.close()

-- INSERT INTO PURCHASE_FROM Table
Inserted 48 purchase records successfully!


In [20]:
# SUPPLIES
conn = pymysql.connect(
    host='127.0.0.1',
    user='skotam',
    password='skotam',
    database='ABCcompany'
)
cursor = conn.cursor()
cursor.execute("DELETE FROM SUPPLIES;")
conn.commit()
print("-- INSERT INTO SUPPLIES Table")
cursor.execute("SELECT VendorID FROM VENDOR")
vendor_ids = [row[0] for row in cursor.fetchall()]
cursor.execute("SELECT PartID FROM PART")
part_ids = [row[0] for row in cursor.fetchall()]
supplies_data = []
supplied_combinations = set()

cup_part_id = 1
for vendor_id in random.sample(vendor_ids, 5):
    last_purchase = fake.date_between(start_date='-1y', end_date='today')
    price = round(random.uniform(5.0, 15.0), 2)
    supplies_data.append((vendor_id, cup_part_id, last_purchase, price))
    supplied_combinations.add((vendor_id, cup_part_id))

for vendor_id in vendor_ids:
    num_parts = random.randint(3, 8)
    parts_to_supply = random.sample(part_ids, min(num_parts, len(part_ids)))
    for part_id in parts_to_supply:
        if (vendor_id, part_id) not in supplied_combinations:
            last_purchase = fake.date_between(start_date='-2y', end_date='today')
            unit_price = round(random.uniform(1.0, 100.0), 2)
            supplies_data.append((vendor_id, part_id, last_purchase, unit_price))
            supplied_combinations.add((vendor_id, part_id))
cursor.executemany(
    "INSERT INTO SUPPLIES (VendorID, PartID, LastPurchaseDate, UnitPrice) VALUES (%s, %s, %s, %s)",
    supplies_data
)
conn.commit()
print(f"Inserted {len(supplies_data)} supply records successfully!")
cursor.close()
conn.close()

-- INSERT INTO SUPPLIES Table
Inserted 114 supply records successfully!


In [21]:
# WORKS_ON
conn = pymysql.connect(
    host='127.0.0.1',
    user='skotam',
    password='skotam',
    database='ABCcompany'
)
cursor = conn.cursor()
cursor.execute("DELETE FROM WORKS_ON;")
conn.commit()
print("-- INSERT INTO WORKS_ON Table")
cursor.execute("SELECT Personal_ID FROM EMPLOYEE")
employee_ids = [row[0] for row in cursor.fetchall()]
cursor.execute("SELECT SiteID FROM SITE")
site_ids = [row[0] for row in cursor.fetchall()]
works_on_data = []
all_site_employees = random.sample(employee_ids, min(3, len(employee_ids)))
for emp_id in all_site_employees:
    for site_id in site_ids:
        works_on_data.append((emp_id, site_id))
for emp_id in employee_ids:
    if emp_id not in all_site_employees:
        num_sites = random.randint(1, 3)
        sites_to_work = random.sample(site_ids, min(num_sites, len(site_ids)))
        for site_id in sites_to_work:
            if (emp_id, site_id) not in [(w[0], w[1]) for w in works_on_data]:
                works_on_data.append((emp_id, site_id))
cursor.executemany(
    "INSERT INTO WORKS_ON (Personal_ID, SiteID) VALUES (%s, %s)",
    works_on_data
)
conn.commit()
print(f"Inserted {len(works_on_data)} work assignments successfully!")
cursor.close()
conn.close()

-- INSERT INTO WORKS_ON Table
Inserted 184 work assignments successfully!


In [22]:
# CREATES
conn = pymysql.connect(
    host='127.0.0.1',
    user='skotam',
    password='skotam',
    database='ABCcompany'
)
cursor = conn.cursor()
cursor.execute("DELETE FROM CREATES;")
conn.commit()
print("-- INSERT INTO CREATES Table")
cursor.execute("SELECT Personal_ID FROM CANDIDATE")
candidate_ids = [row[0] for row in cursor.fetchall()]
cursor.execute("SELECT ProductID FROM PRODUCT")
product_ids = [row[0] for row in cursor.fetchall()]
creates_data = []
for candidate_id in random.sample(candidate_ids, min(20, len(candidate_ids))):
    num_products = random.randint(1, 3)
    products_created = random.sample(product_ids, min(num_products, len(product_ids)))
    for product_id in products_created:
        creates_data.append((candidate_id, product_id))
cursor.executemany(
    "INSERT INTO CREATES (Personal_ID, ProductID) VALUES (%s, %s)",
    creates_data
)
conn.commit()
print(f"Inserted {len(creates_data)} product creation records successfully!")
cursor.close()
conn.close()

-- INSERT INTO CREATES Table
Inserted 39 product creation records successfully!


In [23]:
# TRACKS
conn = pymysql.connect(
    host='127.0.0.1',
    user='skotam',
    password='skotam',
    database='ABCcompany'
)
cursor = conn.cursor()
cursor.execute("DELETE FROM TRACKS;")
conn.commit()
print("-- INSERT INTO TRACKS Table")
cursor.execute("SELECT Personal_ID FROM EMPLOYEE")
employee_ids = [row[0] for row in cursor.fetchall()]
cursor.execute("SELECT Personal_ID FROM CUSTOMER")
customer_ids = [row[0] for row in cursor.fetchall()]
cursor.execute("SELECT ProductID FROM PRODUCT")
product_ids = [row[0] for row in cursor.fetchall()]
tracks_data = []
for _ in range(200):
    employee_id = random.choice(employee_ids)
    customer_id = random.choice(customer_ids)
    product_id = random.choice(product_ids)
    if (employee_id, customer_id, product_id) not in [(t[0], t[1], t[2]) for t in tracks_data]:
        tracks_data.append((employee_id, customer_id, product_id))
cursor.executemany(
    "INSERT INTO TRACKS (Employee_ID, Customer_ID, ProductID) VALUES (%s, %s, %s)",
    tracks_data
)
conn.commit()
print(f"Inserted {len(tracks_data)} tracking records successfully!")
cursor.close()
conn.close()

-- INSERT INTO TRACKS Table
Inserted 200 tracking records successfully!


In [25]:
# SALE_HISTORY
conn = pymysql.connect(
    host='127.0.0.1',
    user='skotam',
    password='skotam',
    database='ABCcompany'
)
cursor = conn.cursor()
cursor.execute("DELETE FROM SALE_HISTORY;")
conn.commit()
print("-- INSERT INTO SALE_HISTORY Table")
cursor.execute("SELECT Personal_ID FROM CUSTOMER")
customer_ids = [row[0] for row in cursor.fetchall()]
cursor.execute("SELECT ProductID, ProductType, ListPrice FROM PRODUCT")
products = cursor.fetchall()
sale_data = []
march_2011_start = datetime(2011, 3, 1)
march_2011_end = datetime(2011, 3, 31, 23, 59, 59)
for _ in range(500):
    customer_id = random.choice(customer_ids)
    product_id, product_type, list_price = random.choice(products)
    if random.random() < 0.8:
        sales_time = fake.date_time_between(start_date='-2y', end_date='now')
        while march_2011_start <= sales_time <= march_2011_end:
            sales_time = fake.date_time_between(start_date='-2y', end_date='now')
    else:
        sales_time = fake.date_time_between(start_date=march_2011_start, end_date=march_2011_end)
    sale_data.append((customer_id, product_id, sales_time))
cursor.executemany(
    "INSERT INTO SALE_HISTORY (Customer_ID, ProductID, SalesTime) VALUES (%s, %s, %s)",
    sale_data
)
conn.commit()
print(f"Inserted {len(sale_data)} sales records successfully!")
cursor.close()
conn.close()

-- INSERT INTO SALE_HISTORY Table
Inserted 500 sales records successfully!


In [26]:
# BUYS
conn = pymysql.connect(
    host='127.0.0.1',
    user='skotam',
    password='skotam',
    database='ABCcompany'
)
cursor = conn.cursor()
cursor.execute("DELETE FROM BUYS;")
conn.commit()
print("-- INSERT INTO BUYS Table")
cursor.execute("SELECT Personal_ID FROM CUSTOMER")
customer_ids = [row[0] for row in cursor.fetchall()]
cursor.execute("SELECT ProductID FROM PRODUCT")
product_ids = [row[0] for row in cursor.fetchall()]
buys_data = []
for _ in range(400):
    customer_id = random.choice(customer_ids)
    product_id = random.choice(product_ids)
    sales_time = fake.date_time_between(start_date='-2y', end_date='now')
    buys_data.append((customer_id, product_id, sales_time))
cursor.executemany(
    "INSERT INTO BUYS (Customer_ID, ProductID, SalesTime) VALUES (%s, %s, %s)",
    buys_data
)
conn.commit()
print(f"Inserted {len(buys_data)} purchase records successfully!")
cursor.close()
conn.close()

-- INSERT INTO BUYS Table
Inserted 400 purchase records successfully!
